# エラーへの対処・修正済みコード

『Statistical Rethinking A Bayesian Course with Examples in R and Stan 』は、  
ベイズ統計学の基礎的な内容を実用例を交えて紹介している非常に優れた教科書です。

教科書には分析で使用するRのコードが記載されていますが、  
著者のRichard McElreath先生のwebサイト  

https://xcelab.net/rm/statistical-rethinking/  

で、 同じ内容を他の言語/パッケージで行うためのGitHubが紹介されています。  
例えばpymcで再現したコードを公開しているリポジトリとして以下があります。  

pymc (https://github.com/pymc-devs/pymc-resources/tree/main)  

notebookの形式で教科書内のコードを(おそらくほぼ)すべて再現されています。  
博愛に満ちた素晴らしい成果物なのですが、一部エラーが出るコードがあります。  

このノートブックではエラーが出たコードの箇所を報告し、  修正できたところは修正済みのコードを記載しています。  

## Chapter5 (Chp_05.ipynb)

#### 5.33, 5.34

元のjupyter notebookのコードでは以降のコードでエラーが生じます。  

    shared_N = shared(dcc["N"].values)
    with pm.Model() as m5_5_draft:
       sigma = pm.Exponential("sigma", 1)
       bN = pm.Normal("bN", 0, 1)
       a = pm.Normal("a", 0, 1)
       mu = pm.Deterministic("mu", a + bN * shared_N)

       K = pm.Normal("K", mu, sigma, observed=dcc["K"])

       m5_5_draft_trace = pm.sample()

    xseq = [-2, 2]
    shared_N.set_value(np.array(xseq))
    with m5_5_draft:
       m5_5_draft_prior_predictive = pm.sample_prior_predictive()  
       
shared_NとKなどの形状が一致せずエラーが発生するようです。  
元のRコードではパッケージの関数を使用しているためどのように処理しているかわりません。  
よって以下ではxseqの形状に合わせてKの形状が変化するように修正しました。

In [ ]:
xseq = np.array([-2, 2])
shared_N.set_value(xseq)

with pm.Model() as m5_5_draft:
    sigma = pm.Exponential("sigma", 1)
    bN = pm.Normal("bN", 0, 1)
    a = pm.Normal("a", 0, 1)
    mu = pm.Deterministic("mu", a + bN * shared_N)

    # ここでobservedの部分を適切な形状に更新
    observed_values = dcc["K"].values[:len(xseq)] 
    K = pm.Normal("K", mu, sigma, observed=observed_values)

    m5_5_draft_trace = pm.sample()


In [ ]:
xseq = [-2, 2]
shared_N.set_value(np.array(xseq))
with m5_5_draft:
    m5_5_draft_prior_predictive = pm.sample_prior_predictive()

#### 5.35  
前のエラーと同様にデータの形状が合わずエラーが出ます。  
同じように修正しました。

In [ ]:
xseq = np.array([-2, 2])
shared_N.set_value(xseq)

with pm.Model() as m5_5:
    sigma = pm.Exponential("sigma", 1)
    bN = pm.Normal("bN", 0, 0.5)
    a = pm.Normal("a", 0, 0.2)
    mu = pm.Deterministic("mu", a + bN * shared_N)

    # ここでobservedの部分を適切な形状に更新
    observed_values = dcc["K"].values[:len(xseq)] 
    K = pm.Normal("K", mu, sigma, observed=observed_values)

    m5_5_trace = pm.sample()
m5_5_data = az.extract_dataset(m5_5_trace)

## Chapter11 (Chp_11.ipynb)

このchapter11では普通に実行するとエラーがかなり出ます。  
なぜなら、古いパッケージのもとで書かれたコードと新しいパッケージのもとで書かれたコードが混在しているからです。  
私はenvironment_v4.ymlの環境で実行しているので、environment.ymlを前提に書かれたコードは一部エラーが出ます。  
そこでエラーの出るコードは、environment_v4.ymlの環境、  
```
Python implementation: CPython
Python version       : 3.9.18
IPython version      : 8.15.0

numpy     : 1.25.2
aesara    : 2.8.7
pymc      : 4.3.0
arviz     : 0.13.0
scipy     : 1.11.2
matplotlib: 3.7.1
pandas    : 2.1.0

Watermark: 2.4.3
```
で実行できるようにコードを書き換えています。

### Code 11.16 and 11.17 
以下のコードでエラーが出ます。
```
with m11_4:
    pm.set_data({"actor_id": np.repeat(range(7), 4), "treat_id": list(range(4)) * 7})
    ppd = pm.sample_posterior_predictive(trace_11_4, random_seed=RANDOM_SEED, var_names=["p"])[
        "posterior_predictive"
    ]["p"]
p_mu = np.array(ppd.mean(["chain", "draw"])).reshape((7, 4))
```
actor_idとtreat_idの型が間違っているようです。  
そのため、actor_idとtreat_idを生成したコードまで遡り、以下のように修正しました。  
以下のコードを実行した後に、コード Code 11.16 and 11.17を実行すると動きます。

In [ ]:
with pm.Model() as m11_4:
    a = pm.Normal("a", 0.0, 1.5, shape=len(actors))
    b = pm.Normal("b", 0.0, 0.5, shape=len(treatments))

    # Create shared variables using pm.MutableData()
    actor_id = pm.Data("actor_id", actor_idx, mutable=True)
    treat_id = pm.Data("treat_id", treat_idx, mutable=True)

    p = pm.Deterministic("p", pm.math.invlogit(a[actor_id] + b[treat_id]))

    pulled_left = pm.Binomial("pulled_left", 1, p, observed=d.pulled_left)
    trace_11_4 = pm.sample(random_seed=RANDOM_SEED)

### Code 11.20
以下のコードを実行するとm11_5が無いことでエラーがでます。  
```
with m11_4:
    pm.set_data({"actor_id": actor_idx, "treat_id": treat_idx})
    trace_11_4 = pm.sample(random_seed=RANDOM_SEED)

az.compare({"m11_4": trace_11_4, "m11_5": trace_11_5})
```
そこで、m11_5を定義するコードを、古いバージョンのnotebookから取り戻して先に実行する必要があります。  
古いバージョンのnotebookへのURL  
https://github.com/pymc-devs/pymc-resources/blob/061ce81c277ac93725beba5bd850a62cd5e81713/Rethinking_2/Chp_11.ipynb  
このnotebookのCode 11.19を先に実行します。

In [ ]:
with pm.Model() as m11_5:
    a = pm.Normal("a", 0.0, 1.5, shape=len(actors))
    bs = pm.Normal("bs", 0.0, 0.5, shape=2)
    bc = pm.Normal("bc", 0.0, 0.5, shape=2)

    p = pm.math.invlogit(a[actor_idx] + bs[side] + bc[cond])

    pulled_left = pm.Binomial("pulled_left", 1, p, observed=d.pulled_left)

    trace_11_5 = pm.sample(random_seed=RANDOM_SEED)